In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# ==== 1. DỮ LIỆU MẪU ====
text = "deep learning transforms the world"
vocab = list(set(text.split()))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for w, i in word2idx.items()}

data = [word2idx[w] for w in text.split()]
inputs  = torch.tensor([data[:-1]])   # ["deep", "learning", "transforms", "the"]
targets = torch.tensor([data[1:]])    # ["learning", "transforms", "the", "world"]

# ==== 2. MÔ HÌNH TRANSFORMER ====
class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=32, nhead=4, num_layers=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x = self.encoder(x)
        out = self.fc(x)
        return out

# ==== 3. KHỞI TẠO, HÀM MẤT MÁT, TỐI ƯU ====
model = TinyTransformer(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ==== 4. HUẤN LUYỆN ===='
for epoch in range(100):
    optimizer.zero_grad()
    output = model(inputs)
    loss = criterion(output.view(-1, len(vocab)), targets.view(-1))
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss = {loss.item():.4f}")

# ==== 5. IN RA DỰ ĐOÁN ====
with torch.no_grad():
    preds = model(inputs)
    predicted_indices = preds.argmax(dim=-1).squeeze().tolist()
    predicted_words = [idx2word[i] for i in predicted_indices]

print("\n=== KẾT QUẢ DỰ ĐOÁN ===")
for i, (inp, pred) in enumerate(zip(inputs.squeeze(), predicted_words)):
    print(f"Từ gốc: {idx2word[inp.item()]:<12}  →  Dự đoán từ kế tiếp: {pred}")


Epoch 20, Loss = 0.0097
Epoch 40, Loss = 0.0020
Epoch 60, Loss = 0.0010
Epoch 80, Loss = 0.0007
Epoch 100, Loss = 0.0007

=== KẾT QUẢ DỰ ĐOÁN ===
Từ gốc: deep          →  Dự đoán từ kế tiếp: learning
Từ gốc: learning      →  Dự đoán từ kế tiếp: transforms
Từ gốc: transforms    →  Dự đoán từ kế tiếp: the
Từ gốc: the           →  Dự đoán từ kế tiếp: world
